### Debt Repayment Plan

#### Part 2 - Advanced Case

- Data
    - **N**  : Net monthly income
    - **M**  : Monthly required savings
    - **I<sub>j</sub>**  : Interest paid on loan to lender **j**
    

- Decision Variables
    - **S<sub>i</sub>**  : Monthly savings during period **i**
    - **P<sub>ij</sub>**  : Debt paid to lender **j** during month **i**
    - **B<sub>ij</sub>**  : Debt owed to lender **j** at the beginning of month **i**

In [1]:
using JuMP
import HiGHS

In [2]:
# Data
data = Dict(
    "lender 1"        => Dict( "loan" => 3200, "term" => 4, "interest" => 3.8 / 100 ),
    "lender 2"        => Dict( "loan" => 4700, "term" => 5, "interest" => 7.3 / 100 ),
    "lender 3"        => Dict( "loan" => 6000, "term" => 6, "interest" => 4.8 / 100 ),
    "lender 4"        => Dict( "loan" => 1500, "term" => 4, "interest" => 3.9 / 100 ),
    "lender 5"        => Dict( "loan" => 2100, "term" => 4, "interest" => 8.1 / 100 ),
    "lender 6"        => Dict( "loan" => 500 , "term" => 3, "interest" => 5.4 / 100 ),
    "lender 7"        => Dict( "loan" => 100 , "term" => 1, "interest" => 3.2 / 100 ),
    "net_income"      => 4500,
    "minimum_monthly_savings" => 1000
);

In [3]:
max_term = maximum(data[loaner]["term"] for loaner in keys(data) if contains(loaner, "lender"))
periods = 1:max_term

LENDERS = sort([loaner for loaner in keys(data) if contains(loaner, "lender")])

7-element Vector{String}:
 "lender 1"
 "lender 2"
 "lender 3"
 "lender 4"
 "lender 5"
 "lender 6"
 "lender 7"

In [4]:
function set_model_variables(
        model::Model, 
        periods::UnitRange, 
        NET_INCOME::Number, 
        MINIMUM_SAVINGS::Number,
        LENDERS::Array{String}
    )
    # Monthly savings
    Savings = @variable(model, Saving[periods] >= MINIMUM_SAVINGS );

    # Monthly debt reimbursed to each lender
    Payments = @variable(model, Payment[periods, LENDERS] >= 0);

    # Monthly balance due to each lender
    Balances = @variable(model, Balance[cat(periods, periods[end] + 1, dims = 1), LENDERS] >= 0);
    
    return Savings, Payments, Balances
end

set_model_variables (generic function with 1 method)

In [5]:
function set_model_constraints( 
        model::Model, 
        debt::Dict,
        NET_INCOME::Int,
        Savings::JuMP.Containers.DenseAxisArray, 
        Payments::JuMP.Containers.DenseAxisArray, 
        Balances::JuMP.Containers.DenseAxisArray
    )
    loan_label = "loan"; period_label = "term"; interest_label = "interest";

    # Debt owed at the beginning of period 1 to each lender
    @constraint( model, [lender in keys(debt)], Balances[1, lender] == debt[lender][loan_label] )

    # Ensure that all debt is paid off by the grace period
    @constraint( model, [lender in keys(debt)] , Balances[debt[lender][period_label] + 1, lender] == 0 )

    # Debt owed the beginning of period 2...end
    @constraint( model, 
        [i in periods, lender in keys(debt)], 
        Balances[i + 1, lender] == (1 + debt[lender][interest_label]) * (Balances[i, lender] - Payments[i, lender]) 
    )

    # Monthly expense can't exceed net income
    @constraint( model, [i in periods], Savings[i] + sum(Payments[i, LENDERS]) <= NET_INCOME )

end

set_model_constraints (generic function with 1 method)

#### Solve Model

In [16]:
model = Model( HiGHS.Optimizer )

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [17]:
Savings, Payments, Balance = set_model_variables(model, periods, data["net_income"], data["minimum_monthly_savings"], LENDERS)

(1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:6
And data, a 6-element Vector{VariableRef}:
 Saving[1]
 Saving[2]
 Saving[3]
 Saving[4]
 Saving[5]
 Saving[6], 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 1:6
    Dimension 2, ["lender 1", "lender 2", "lender 3", "lender 4", "lender 5", "lender 6", "lender 7"]
And data, a 6×7 Matrix{VariableRef}:
 Payment[1,lender 1]  Payment[1,lender 2]  …  Payment[1,lender 7]
 Payment[2,lender 1]  Payment[2,lender 2]     Payment[2,lender 7]
 Payment[3,lender 1]  Payment[3,lender 2]     Payment[3,lender 7]
 Payment[4,lender 1]  Payment[4,lender 2]     Payment[4,lender 7]
 Payment[5,lender 1]  Payment[5,lender 2]     Payment[5,lender 7]
 Payment[6,lender 1]  Payment[6,lender 2]  …  Payment[6,lender 7], 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7]
    Dimension 2, ["lender 1", "lender 2", "lender 3", "lender 4", "lender 5",

In [18]:
debt = Dict(lender => data[lender] for lender in LENDERS)
set_model_constraints( model, debt, data["net_income"], Savings, Payments, Balance )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:6
And data, a 6-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 Saving[1] + Payment[1,lender 1] + Payment[1,lender 2] + Payment[1,lender 3] + Payment[1,lender 4] + Payment[1,lender 5] + Payment[1,lender 6] + Payment[1,lender 7] <= 4500.0
 Saving[2] + Payment[2,lender 1] + Payment[2,lender 2] + Payment[2,lender 3] + Payment[2,lender 4] + Payment[2,lender 5] + Payment[2,lender 6] + Payment[2,lender 7] <= 4500.0
 Saving[3] + Payment[3,lender 1] + Payment[3,lender 2] + Payment[3,lender 3] + Payment[3,lender 4] + Payment[3,lender 5] + Payment[3,lender 6] + Payment[3,lender 7] <= 4500.0
 Saving[4] + Payment[4,lender 1] + Payment[4,lender 2] + Payment[4

In [19]:
@objective(model, Max, sum(Savings))

Saving[1] + Saving[2] + Saving[3] + Saving[4] + Saving[5] + Saving[6]

In [20]:
print(model)

In [21]:
function solve_infeasible(model)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        return objective_value(model)
    else
        @warn("The model was not solved correctly.")
        return nothing
    end
end

solve_infeasible(model)

Presolving model
20 rows, 40 cols, 74 nonzeros
11 rows, 31 cols, 56 nonzeros
11 rows, 31 cols, 56 nonzeros
Presolve : Reductions: rows 11(-51); columns 31(-66); elements 56(-132)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.9999922038e+00 Ph1: 5(5); Du: 5(4.99999) 0s
         22    -6.8379122112e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 22
Objective value     :  6.8379122112e+03
HiGHS run time      :          0.00


6837.912211234662

In [22]:
using Printf
function print_payment_plan( 
        model::Model,
        debt::Dict,
        Savings::JuMP.Containers.DenseAxisArray, 
        Payments::JuMP.Containers.DenseAxisArray, 
        Balances::JuMP.Containers.DenseAxisArray
    )
    @printf("| %-10s | %-10s | %-10s | %-10s | %-10s | %-10s | %-10s | %-12s | %5s |\n",
        "Period", "Lender", "Balance", "Payment", "Savings", 
        "Rate (%)", "Interest", "End Balance", "Term"
    )
    cumulative_savings = cumsum([value(Savings[i]) for i in periods])
    for i in periods
        first_print = true
        for lender in LENDERS
            start_balance = value(Balances[i, lender])
            end_balance = value(Balances[i, lender]) - value(Payments[i, lender])
            payment = value(Payments[i, lender])
            interest = ( 
                i == 1 ? 0 
                : 
                value(Balances[i, lender]) - (value(Balances[i - 1, lender] ) - value(Payments[i - 1, lender])) 
            )
            @printf("| %10s | %-10s | %10.2f | %10.2f | %10s | %10.2f | %10.2f | %12.2f | %5s |\n", 
                if first_print i else "-" end,
                lender,
                start_balance > 0 ? start_balance : 0.00,
                payment > 0 ? payment : 0.00, 
                "-",
                debt[lender]["interest"],
                interest > 0 ? interest : 0.00,
                end_balance > 0 ? end_balance : 0.00,
                debt[lender]["term"]
            )
            first_print = false
        end
        @printf("| %10s | %-10s | %10s | %10s | %10.2f | %10s | %10s | %12s | %5s |\n", 
                "-",
                "-",
                "-", 
                "-", 
                cumulative_savings[i],
                "-",
                "-",
                "-",
                "-"
            )
    end
end;

In [23]:
if termination_status(model) == OPTIMAL
    print_payment_plan(model, debt, Savings, Payments, Balance)
end

| Period     | Lender     | Balance    | Payment    | Savings    | Rate (%)   | Interest   | End Balance  |  Term |
|          1 | lender 1   |    3200.00 |       0.00 |          - |       0.04 |       0.00 |      3200.00 |     4 |
|          - | lender 2   |    4700.00 |    1300.00 |          - |       0.07 |       0.00 |      3400.00 |     5 |
|          - | lender 3   |    6000.00 |       0.00 |          - |       0.05 |       0.00 |      6000.00 |     6 |
|          - | lender 4   |    1500.00 |       0.00 |          - |       0.04 |       0.00 |      1500.00 |     4 |
|          - | lender 5   |    2100.00 |    2100.00 |          - |       0.08 |       0.00 |         0.00 |     4 |
|          - | lender 6   |     500.00 |       0.00 |          - |       0.05 |       0.00 |       500.00 |     3 |
|          - | lender 7   |     100.00 |     100.00 |          - |       0.03 |       0.00 |         0.00 |     1 |
|          - | -          |          - |          - |    1000.00 |      

In [24]:
@printf("%119s", '\U2111')

                                                                                                                      ℑ